# Backtrader For ASX Mining Stocks 

### Target Stocks And Timeframes

In [2]:
# Dependency

import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
from pandas_datareader import data
from datetime import datetime as dt
import numpy as np
import backtrader as bt



# Use 20 years as the time period for getting best SMA dyas to create a trading strategy

start_date1 = dt(2000,1,1)

end_date1 = dt(2019,12,31)

# testing the strategy by using 2 years known period 

start_date2 = dt(2020,1,1)

end_date2 = dt(2022,9,21)


# Use yfinance to extract data

yf.pdr_override()

# Target the 3 large mining companies in ASX

targets = ['RIO.AX','FMG.AX','BHP.AX']

#Transform data to csv files

for i in range(0,len(targets)):    

    stock = data.get_data_yahoo([targets[i]], start_date1, end_date2)

    path = f'InputData/{targets[i]}.csv'

    record = stock.to_csv(path)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Maximize The Earning

In [4]:

# Use for loop to get the best match of short SMA and long SMA 

rangeShort=np.arange(5,81,1)

rangeLong=np.arange(100,120,1)


bestMatch=[] 

for u in range(0,len(targets)):  

    testBestPeriod=[] 

    for i in rangeShort:

        for j in rangeLong:

            class SMAStrategy(bt.Strategy):

                # Initiate the backtest 

                def __init__(self):

                    self.dataclose = self.data0.close

                    self.order = None

                    self.buyprice = None
                    
                    self.buycomm = None
                
                    self.smaS = bt.indicators.SMA(self.data0,period=i)
                    self.smaL = bt.indicators.SMA(self.data0,period=j)
                    
                    

                def next(self):
                    if not self.position:
                        if self.smaS[0]>self.smaL[0] :
                            self.buy()
                    else:
                        if self.smaS[0]<self.smaL[0] :    
                                self.close()

            cerebro=bt.Cerebro()

            df=pd.read_csv(f'InputData/{targets[u]}.csv',header=0, index_col='Date', parse_dates=['Date'])

            testStock=bt.feeds.PandasData(dataname=df,fromdate=start_date1, todate=end_date1,timeframe=bt.TimeFrame.Days)

            cerebro.adddata(testStock)

            cerebro.addstrategy(SMAStrategy)

            cerebro.broker.setcash(10000)

            cerebro.broker.setcommission(commission=0.002)

            cerebro.addsizer(bt.sizers.PercentSizer, percents=90) # Use 90% cash for trading
            
            result = cerebro.run()

            portvalue = cerebro.broker.getvalue()

            match={'shortPeriod':i,'longPeriod':j,'finalValue':portvalue}

            testBestPeriod.append(match)

    max_value = max(testBestPeriod,key=lambda x:x['finalValue'])

    bestShort=max_value['shortPeriod']

    bestLong=max_value['longPeriod']

    class SMAStrategy(bt.Strategy):

        def __init__(self):

            self.dataclose = self.data0.close

            self.order = None

            self.buyprice = None

            self.buycomm = None
        
            self.smaS = bt.indicators.SMA(self.data0,period=bestShort)

            self.smaL = bt.indicators.SMA(self.data0,period=bestLong)
            
            

        def next(self):

            if not self.position:

                if self.smaS[0]>self.smaL[0] :

                    self.buy()
            else:
                if self.smaS[0]<self.smaL[0] : 

                    self.close()
    
    cerebro=bt.Cerebro()

    df=pd.read_csv(f'InputData/{targets[u]}.csv',header=0, index_col='Date', parse_dates=['Date'])

    testStock=bt.feeds.PandasData(dataname=df,fromdate=start_date2, todate=end_date2,timeframe=bt.TimeFrame.Days)

    cerebro.adddata(testStock)

    cerebro.addstrategy(SMAStrategy)

    cerebro.broker.setcash(10000)

    cerebro.broker.setcommission(commission=0.002)

    cerebro.addsizer(bt.sizers.PercentSizer, percents=90) # Use 90% cash for trading

    cerebro.run()

    result = cerebro.broker.getvalue()

    bestMatch.append({'Stock':targets[u],'shortPeriod':bestShort,'longPeriod':bestLong,'finalValue':round(result,2),'ROI':f'{round(((result-10000)/10000*100),2)}%'})

    plt.rcParams['figure.figsize'] = [15, 12]

    plt.rcParams.update({'font.size': 12}) 

    img=cerebro.plot(iplot=False)

    img[0][0].savefig(f'OutputData/{targets[u]}.png')


### Result

In [5]:
print(bestMatch[0])

print(bestMatch[1])

print(bestMatch[2])

{'Stock': 'RIO.AX', 'shortPeriod': 62, 'longPeriod': 104, 'finalValue': 9040.01, 'ROI': '-9.6%'}
{'Stock': 'FMG.AX', 'shortPeriod': 30, 'longPeriod': 109, 'finalValue': 12176.39, 'ROI': '21.76%'}
{'Stock': 'BHP.AX', 'shortPeriod': 34, 'longPeriod': 101, 'finalValue': 9657.47, 'ROI': '-3.43%'}


In [6]:
Result = pd.DataFrame(bestMatch)

Result

Result.to_csv('OutputData/ResultMiningROI.csv')

# Merge Results

In [22]:
from turtle import left

# Import two csv

Mine_df = pd.read_csv('OutputData/ResultMiningROI.csv')

Bank_df = pd.read_csv('OutputData/ResultBankROI.csv')

# Merge two Dataframe

Final_df = Mine_df.merge(Bank_df, how='outer')

# Drop unwanted collumn

Cleaned_df = Final_df.drop(columns='Unnamed: 0')

Cleaned_df


,Stock,shortPeriod,longPeriod,finalValue,ROI
0,RIO.AX,62,104,9040.01,-9.6%
1,FMG.AX,30,109,12176.39,21.76%
2,BHP.AX,34,101,9657.47,-3.43%
3,NAB.AX,60,119,13886.80,38.87%
4,CBA.AX,47,101,9847.04,-1.53%
5,ANZ.AX,78,116,12854.82,28.55%
6,WBC.AX,64,119,10920.40,9.2%


In [32]:
# Export Cleaned_df as csv 

Cleaned_df.to_csv('OutputData/TestResult.csv')






AttributeError: 'PlotAccessor' object has no attribute 'table'